# Hydrometric and Water Quality Data Integration

The goal of this notebook is to integrate data acquired so far. Right now we have hydrometric and water quality data.

In [31]:
import numpy as np
import pandas as pd

## 1. Hydrometric

We have 7 hydrometric stations with data from 2010 to 2018. For this step, we have to make sure all the data is syncronized by the datetime. We need to get all the datetimes used on Hydrometric data.

In [183]:
hydrometric = pd.read_csv('hydrometric_data_daily_2010_2018.csv', sep=',')
hydrometric.head()

,ID,PARAM,YEAR,DD,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,02HB001,2,2010,1,1.513,1.483,1.481,1.552,1.500,1.468,1.576,1.510,1.489,1.547,1.496,1.678
1,02HB001,2,2010,2,1.513,1.478,1.480,1.561,1.507,1.492,1.551,1.503,1.492,1.532,1.501,1.620
2,02HB001,2,2010,3,1.516,1.480,1.479,1.559,1.595,1.680,1.530,1.501,1.537,1.523,1.500,1.563
3,02HB001,2,2010,4,1.504,1.477,1.477,1.553,1.554,1.605,1.516,1.501,1.573,1.512,1.502,1.539
4,02HB001,2,2010,5,1.498,1.477,1.473,1.546,1.526,1.543,1.508,1.511,1.536,1.503,1.508,1.523


We need to convert the corrent structure so we can make a datetime column being the unique key of our dataframe.

In [184]:
from datetime import datetime

# Melt DataFrame
hydrometric_melted = hydrometric.melt(id_vars=['ID', 'PARAM', 'YEAR', 'DD'], var_name='Month', value_name='WATER_LEVEL')

# Map month names to Numbers
month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}
hydrometric_melted['Month'] = hydrometric_melted['Month'].map(month_map)

# Validate dates
def valid_date(year, month, day):
    try:
        return datetime(year, month, day)
    except ValueError:
        return None

# Create datetime columns
hydrometric_melted['DATE_YYYYMMDD'] = hydrometric_melted.apply(lambda row: valid_date(row['YEAR'], int(row['Month']), row['DD']), axis=1)

# Remove null rows
hydrometric_melted = hydrometric_melted.dropna(subset=['WATER_LEVEL'])

# Select only the important columns
hydrometric_final = hydrometric_melted[['ID', 'DATE_YYYYMMDD', 'WATER_LEVEL']]
hydrometric_final.rename(columns={'ID': 'STATION', 'DATE_YYYYMMDD': 'DATE'}, inplace=True)

hydrometric_final.head()

C:\Users\IEP User 4\AppData\Local\Temp\ipykernel_13272\4114671135.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydrometric_final.rename(columns={'ID': 'STATION', 'DATE_YYYYMMDD': 'DATE'}, inplace=True)


,STATION,DATE,WATER_LEVEL
0,02HB001,2010-01-01,1.513
1,02HB001,2010-01-02,1.513
2,02HB001,2010-01-03,1.516
3,02HB001,2010-01-04,1.504
4,02HB001,2010-01-05,1.498


In [305]:
hydrometric_pivot = hydrometric_final.pivot(index='DATE', columns='STATION', values='WATER_LEVEL')
hydrometric_pivot.columns.name = None
hydrometric_pivot.reset_index(inplace=True)
hydrometric_pivot.head()

,DATE,02HB001,02HB008,02HB013,02HB018,02HB025,02HB029,02HB031
0,2010-01-01,1.513,2.233,11.216,3.919,2.676,4.524,2.032
1,2010-01-02,1.513,2.206,11.212,3.912,2.684,4.723,2.065
2,2010-01-03,1.516,2.212,11.210,3.983,2.751,5.239,2.091
3,2010-01-04,1.504,2.217,11.211,4.025,2.806,5.408,2.016
4,2010-01-05,1.498,2.216,11.209,4.042,2.823,5.364,2.020


## 2. Water Quality

In [296]:
quality = pd.read_csv('water_quality_separated_data_2010-2018.csv', sep=',').drop(['TIME_HH:MM'], axis='columns')
quality[['STATION', 'DATE_YYYYMMDD', 'RESULT']].head()

,STATION,DATE_YYYYMMDD,RESULT
0,6007600302,2010-01-06,21.300
1,6007600302,2010-01-06,97.600
2,6007600302,2010-01-06,0.038
3,6007600302,2010-01-06,3.000
4,6007600302,2010-01-06,89.500


Here we are going to rename our columns to facilitate data analysis 

In [297]:
def get_parm(parm):
    return parm['PARM_DESCRIPTION'].split(',')[0]

quality['PARM'] = quality.apply(get_parm, axis=1)

In [298]:
quality_renamed = quality.rename(columns={'DATE_YYYYMMDD': 'DATE'}).drop(['PARM_DESCRIPTION', 'UNITS'], axis=1)
quality_renamed.head()

,STATION,DATE,RESULT,PARM
0,6007600302,2010-01-06,21.300,MAGNESIUM
1,6007600302,2010-01-06,97.600,CHLORIDE
2,6007600302,2010-01-06,0.038,AMMONIUM
3,6007600302,2010-01-06,3.000,NITRATES TOTAL
4,6007600302,2010-01-06,89.500,CALCIUM


Now we need to remove duplicate values. It happend twice the same station measuring two times a day water quality. So for that cases we are going to get the mean of the values.

In [299]:
quality_pivot = quality_renamed.groupby(['STATION', 'DATE', 'PARM']).mean()
quality_pivot = quality_pivot.pivot_table(index='DATE', columns=['STATION', 'PARM'], values='RESULT').stack().reset_index()
quality_pivot.head()

STATION,DATE,PARM,6007600302,6007600602,6007601002,6007601702,6007601802,6007602302,6007605002
0,2010-01-06,AMMONIUM,0.038,0.089,0.044,0.026,0.06,0.121,0.029
1,2010-01-06,CALCIUM,89.500,87.800,83.100,92.700,80.30,87.400,99.200
2,2010-01-06,CHLORIDE,97.600,270.000,68.500,98.300,105.00,240.000,218.000
3,2010-01-06,MAGNESIUM,21.300,21.800,19.500,21.400,19.50,20.500,22.200
4,2010-01-06,NITRATES TOTAL,3.000,1.330,1.580,2.630,1.80,1.780,2.780


In [300]:
value_columns = quality_pivot.columns[3:]
quality_pivot = quality_pivot.pivot(index=['DATE'], columns='PARM', values=value_columns).reset_index()
quality_pivot.head()

STATION        DATE 6007600602                                            \
PARM                  AMMONIUM CALCIUM CHLORIDE MAGNESIUM NITRATES TOTAL   
0        2010-01-06      0.089    87.8    270.0      21.8          1.330   
1        2010-02-24      0.141    98.8    371.0      24.0          1.540   
2        2010-03-31      0.066    90.0    350.0      19.3          1.390   
3        2010-04-28      0.027    84.1    339.0      20.0          0.744   
4        2010-05-26      0.071    75.8    285.0      21.0            NaN   

STATION                  6007601002          ...     6007602302            \
PARM    POTASSIUM SODIUM   AMMONIUM CALCIUM  ... NITRATES TOTAL POTASSIUM   
0             NaN    NaN      0.044    83.1  ...          1.780       NaN   
1             NaN    NaN      0.155    81.8  ...          1.790       NaN   
2            5.99  188.0      0.019    71.4  ...          1.190      4.25   
3            5.05  160.0      0.009    79.6  ...          0.774      3.95   
4            5.89  166.0      0.031    76.0  ...            NaN      4.99   

STATION        6007605002                                                      \
PARM    SODIUM   AMMONIUM CALCIUM CHLORIDE MAGNESIUM NITRATES TOTAL POTASSIUM   
0          NaN      0.029    99.2    218.0      22.2           2.78       NaN   
1          NaN      0.033    94.3    599.0      21.6           2.94       NaN   
2        130.0      0.011    75.4    122.0      18.6           1.59      2.64   
3        132.0      0.011    77.4    107.0      21.5           1.79      2.70   
4        141.0      0.051    71.3    115.0      21.1            NaN      3.42   

STATION         
PARM    SODIUM  
0          NaN  
1          NaN  
2         72.2  
3         69.6  
4         67.4  

[5 rows x 43 columns]

In [301]:
quality_pivot.reset_index().head()

STATION index        DATE 6007600602                             \
PARM                        AMMONIUM CALCIUM CHLORIDE MAGNESIUM   
0           0  2010-01-06      0.089    87.8    270.0      21.8   
1           1  2010-02-24      0.141    98.8    371.0      24.0   
2           2  2010-03-31      0.066    90.0    350.0      19.3   
3           3  2010-04-28      0.027    84.1    339.0      20.0   
4           4  2010-05-26      0.071    75.8    285.0      21.0   

STATION                                 6007601002  ...     6007602302  \
PARM    NITRATES TOTAL POTASSIUM SODIUM   AMMONIUM  ... NITRATES TOTAL   
0                1.330       NaN    NaN      0.044  ...          1.780   
1                1.540       NaN    NaN      0.155  ...          1.790   
2                1.390      5.99  188.0      0.019  ...          1.190   
3                0.744      5.05  160.0      0.009  ...          0.774   
4                  NaN      5.89  166.0      0.031  ...            NaN   

STATION                  6007605002                                            \
PARM    POTASSIUM SODIUM   AMMONIUM CALCIUM CHLORIDE MAGNESIUM NITRATES TOTAL   
0             NaN    NaN      0.029    99.2    218.0      22.2           2.78   
1             NaN    NaN      0.033    94.3    599.0      21.6           2.94   
2            4.25  130.0      0.011    75.4    122.0      18.6           1.59   
3            3.95  132.0      0.011    77.4    107.0      21.5           1.79   
4            4.99  141.0      0.051    71.3    115.0      21.1            NaN   

STATION                   
PARM    POTASSIUM SODIUM  
0             NaN    NaN  
1             NaN    NaN  
2            2.64   72.2  
3            2.70   69.6  
4            3.42   67.4  

[5 rows x 44 columns]

In [302]:
quality_pivot.columns = [f'{col[1]}_{col[0]}' if col[0] != '' else col[1] for col in quality_pivot.columns]
quality_pivot = quality_pivot.rename(columns={'_DATE':'DATE'})

## 3. Integration

In [331]:
def cvt2date(row):
    split = row.split('-')
    return datetime(int(split[0]), int(split[1]), int(split[2]))

In [332]:
merged_df = pd.merge(hydrometric_pivot, quality_pivot, on='DATE', how='left')

In [333]:
merged_df.sort_values(by='DATE')

,DATE,02HB001,02HB008,02HB013,02HB018,02HB025,02HB029,02HB031,AMMONIUM_6007600602,CALCIUM_6007600602,...,NITRATES TOTAL_6007602302,POTASSIUM_6007602302,SODIUM_6007602302,AMMONIUM_6007605002,CALCIUM_6007605002,CHLORIDE_6007605002,MAGNESIUM_6007605002,NITRATES TOTAL_6007605002,POTASSIUM_6007605002,SODIUM_6007605002
0,2010-01-01,1.513,2.233,11.216,3.919,2.676,4.524,2.032,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-02,1.513,2.206,11.212,3.912,2.684,4.723,2.065,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-03,1.516,2.212,11.210,3.983,2.751,5.239,2.091,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,1.504,2.217,11.211,4.025,2.806,5.408,2.016,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-05,1.498,2.216,11.209,4.042,2.823,5.364,2.020,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282,2018-12-27,1.375,2.231,11.201,3.922,2.710,4.517,1.917,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3283,2018-12-28,1.491,2.283,11.278,4.079,2.804,4.634,1.956,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3284,2018-12-29,1.510,2.304,11.249,4.193,2.885,4.719,1.932,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3285,2018-12-30,1.432,2.290,11.215,4.056,2.811,4.638,1.922,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
